In [5]:
!pip install dash

     |████████████████████████████████| 1.1 MB 16.2 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 8.7 MB/s  eta 0:00:01
     |████████████████████████████████| 3.7 MB 68.4 MB/s eta 0:00:01
     |████████████████████████████████| 83 kB 6.2 MB/s  eta 0:00:01
     |████████████████████████████████| 1.8 MB 58.5 MB/s eta 0:00:01
     |████████████████████████████████| 288 kB 52.4 MB/s eta 0:00:01
     |████████████████████████████████| 357 kB 59.3 MB/s eta 0:00:01
  Created wheel for dash: filename=dash-1.21.0-py3-none-any.whl size=1100478 sha256=e39c66d12b15cd518c5084f2b9d1a2bffb75fb74addbb06514376982613f98c6
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/56/e4/21/8a1190a7c31f3aa3f8834b05126ded8bf105b8fd11fad1806e
  Created wheel for dash-core-components: filename=dash_core_components-1.17.1-py3-none-any.whl size=3738983 sha256=efd69d9a42a808a965740ff3cca8bee126701633717712c9956b5477b3ce5d46
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/08/ae/6a/0f4a0ee8fa216e

In [ ]:
# Import required libraries
import wget
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px


9600.0
0.0
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


In [ ]:

# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
spacex_df=pd.read_csv(spacex_csv_file)

spacex_df.head()

x=spacex_df.groupby('Launch Site')['class'].count()
pd.DataFrame(x)

x=spacex_df.groupby('Launch Site')['class'].count()
pd.DataFrame(x)

class_breakdown=spacex_df.groupby(['Launch Site','class'])['class'].count()
class_breakdown=pd.DataFrame(class_breakdown)
class_breakdown.columns=['Count']
class_breakdown=class_breakdown.reset_index()
class_breakdown[class_breakdown["Launch Site"]=="CCAFS LC-40"]


data_class=class_breakdown[class_breakdown["Launch Site"]=="CCAFS LC-40"]

pie1=px.pie(data_frame=data_class,names="class",values="Count")
#pie1.update_layout(legend=dict(ticks='inside',tickvals=[0, 1],ticktext=["Failure","Success"]),legend_title_text='Launch Outcome')

# Read the airline data into pandas dataframe
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

px.scatter(data_frame=spacex_df,x='Payload Mass (kg)',y='Mission Outcome')

print(max_payload)
print(min_payload)

mark_values={0:'0',600:'600',1200:'1200',1800:'1800',2400:'2400',3000:'3000',3600:'3600',
             4200:'4200',4800:'4800',5400:'5400',6000:'6000', 6600:'6600', 7200:'7200',
             7800:'7800',8400:'8400',9000:'9000',9600:'9600'}

# you need to include __name__ in your Dash constructor if
# you plan to use a custom CSS or JavaScript in your Dash apps
app = dash.Dash(__name__)

# ---------------------------------------------------------------
app.layout = html.Div([html.H1('SpaceX Launch Records Dashboard',
                               style={'textAlign': 'center', 'color': '#503D36',
                                      'font-size': 40}),

                       html.Div([
                           html.Label(['Launch Site Selection']),
                           dcc.Dropdown(
                               id='site_dropdown',
                               options=[
                                   {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                   {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                   {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                   {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                   {'label': 'All Sites', 'value': 'All'}
                               ],
                               value='All',
                               placeholder="Select a Launch Site here",
                               searchable=True,
                               style={"width": "50%"}
                           ),
                       ]),

                       # TASK 2: Add a pie chart to show the total successful launches count for all sites
                       # If a specific launch site was selected, show the Success vs. Failed counts for the site
                       html.Div([
                           dcc.Graph(id='the_graph')
                       ]),

                       # TASK 3: Add a slider to select payload range
                       html.Div([
                           html.Label(["Payload range (Kg):"],
                                      style={'font-weight': 'bold'}),

                           html.P(),

                           dcc.RangeSlider(id='my_range_slider', min=min_payload, max=max_payload, marks=mark_values,
                                           value=[min_payload, max_payload], step=600)
                       ]),
                       # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                       html.Div([
                           dcc.Graph(id='success_payload_scatter_chart')
                       ]),

                       ])


# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output


# ---------------------------------------------------------------
@app.callback(
    [Output(component_id='the_graph', component_property='figure'),
     Output(component_id='success_payload_scatter_chart', component_property='figure')],
    [Input(component_id='site_dropdown', component_property='value'),
     Input(component_id='my_range_slider', component_property='value')]
)
def update_graph(site_dropdown, my_range_slider):
    dff = spacex_df

    class_breakdown = pd.DataFrame(dff.groupby(['Launch Site', 'class'])['class'].count())
    class_breakdown.columns = ['Count']
    class_breakdown = class_breakdown.reset_index()
    # df_list=dff[dff["Launch Site"]==site_dropdown]

    if site_dropdown == 'All':
        dff_filtered = dff
        piechart = px.pie(data_frame=dff_filtered, names="Launch Site", values="class",
                          title='Total Successful Launches by Site')
        piechart.update_traces(textinfo='value')

    else:
        class_breakdown_filtered = class_breakdown[class_breakdown["Launch Site"] == site_dropdown]
        dff_filtered = dff[dff["Launch Site"] == site_dropdown]
        ###List of Categories in Data
        # df_list=dff[dff["Launch Site"].isin(site_dropdown)]
        piechart = px.pie(data_frame=class_breakdown_filtered, names="class", values="Count",
                          title='Total Success Launches for Site: ' + site_dropdown)

    ###Note: Output from range is list [x,y]
    dff_range = dff_filtered[(dff_filtered['Payload Mass (kg)'] >= my_range_slider[0]) & (
                dff_filtered['Payload Mass (kg)'] <= my_range_slider[1])]
    scatter_plot = px.scatter(data_frame=dff_range, x='Payload Mass (kg)', y='class', color="Booster Version Category",
                              opacity=.4, title='Correlation between Payload and Success for: ' + site_dropdown)
    ###List of Locals

    return (piechart, scatter_plot)


# Run the app
if __name__ == '__main__':
    app.run_server()






